# Add Configuration attribute to Teleview

This notebook Helps you add a configuration attribute to the Teleview Project.

So far it works only for adding boolean config parameters


## Setup Variables

In [1]:
persist_h=r'..\persist.h'
telegramUtils_h=r'..\telegram_utils.h'
webPages_h=r'..\webPages.h'

# strings will be added after this attribute
anchor_attribute="vFlip"


In [2]:
new_attribute_name="useBuzzer"
new_attribute_type="boolean"
new_attribute_desc="Buzzer on motion Detect"

## Changes in persist.h

In [3]:
import re 

def changePersist(anchor_attribute,new_attribute_name,new_attribute_type,new_attribute_desc):
    with open(persist_h,"r") as persistH:
        lines=[]
        for line in persistH:
            
            ############################
            #boolean vFlip;
            match1 = re.search(F"boolean {anchor_attribute};", line)
            
            #.vFlip = true,
            match2 = re.search(F".{anchor_attribute} = true,", line)
            
            # prefs.remove("vFlip");
            #match3 = re.search(F'prefs.remove(\"{anchor_attribute}\");', line)
            match3 = re.search(f'prefs.remove\("vFlip"\);', line)

            # ci.vFlip = prefs.getBool("vFlip",configItems.vFlip);
            match4 = re.search(F'ci.{anchor_attribute} = prefs.getBool\("{anchor_attribute}",configItems.{anchor_attribute}\);', line)
            
            # if (prefs.getBool("vFlip")!=ci->vFlip)
            #   { prefs.putBool("vFlip",ci->vFlip); bDirty=true; }
            match5 = re.search(F'   {{ prefs.putBool\("{anchor_attribute}",ci->{anchor_attribute}\); bDirty=true; }}', line)
            
            #   result += prefix+"vFlip            "+sep+"";
            #   result += (ci->vFlip ? String("true") : String("false"))  + suffix;
            #match6 = re.search(F'result += \(ci->{anchor_attribute} ? String\("true"\) : String\("false"\)\)  + suffix;', line)
            escaped= re.escape(F'result += (ci->{anchor_attribute} ? String("true") : String("false"))  + suffix;')
            #print ("escaped",escaped)
            match6 = re.search(escaped, line)
            ############################
            lines.append(line)
            new_line=""
            new_line2=""
            if match1:
                new_line=F"  {new_attribute_type} {new_attribute_name};"
            if match2:
                new_line=F"  .{new_attribute_name} = true,"
            if match3:
                new_line=F"    prefs.remove(\"{new_attribute_name}\");"
            if match4:
                new_line=F"    ci.{new_attribute_name} = prefs.getBool(\"{new_attribute_name}\",configItems.{new_attribute_name});"
            if match5:
                new_line =F'    if (prefs.getBool("{new_attribute_name}")!=ci->{new_attribute_name})'
                new_line2=F'      {{ prefs.putBool("{new_attribute_name}",ci->{new_attribute_name}); bDirty=true; }}'
            if match6:
                new_line =F'  result += prefix+"{new_attribute_name}            "+sep+"";'
                new_line2=F'  result += (ci->{new_attribute_name} ? String("true") : String("false"))  + suffix;'
            ############################
            if new_line!="":
                print (new_line)
                lines.append(new_line+"\n")
            if new_line2!="":
                print (new_line2)
                lines.append(new_line2+"\n")
            ############################
        new_persist_h=open(F"{persist_h}_new","w")
        for line in lines:
            #print (line)
            new_persist_h.write(line)
        new_persist_h.close()

    if os.path.exists(F"{persist_h}_orig"):
        os.unlink(F"{persist_h}_orig")
    os.rename(persist_h,F"{persist_h}_orig")
    os.rename(F"{persist_h}_new",persist_h)

## Changes in telegram_utils.h

In [4]:
import re 
import os

def changeTelegram_utils(anchor_attribute,new_attribute_name,new_attribute_type,new_attribute_desc):
    with open(telegramUtils_h,"r") as telegramUtilsH:
        lines=[]
        for line in telegramUtilsH:
            #print (line)
            ############################
            #keyboardJson += "[{ \"text\" : \"Camera Flip is ";
            #keyboardJson += (configItems.vFlip?"ON\u2705":"OFF\u274C");
            #keyboardJson += "\", \"callback_data\" : \"/vFlip\" }],";
            lookfor=r'keyboardJson += "\", \"callback_data\" : \"/vFlip\" }],";'
            lookfor=lookfor.replace("vFlip",anchor_attribute)
            match1 = re.search(re.escape(lookfor), line)

            # }else if (text == "/vFlip") {
            #    configItems.vFlip = !configItems.vFlip;
            match2 = re.search("configItems.vFlip = !configItems.vFlip;".replace("vFlip",anchor_attribute), line)

            #welcome += "\t vFlip       | Camera vertical FLIP\n";
            lookfor=r'welcome += "\t vFlip       | Camera vertical FLIP\n";'
            lookfor=lookfor.replace("vFlip",anchor_attribute)
            match3 = re.search(re.escape(lookfor), line)
            
            ############################
            lines.append(line)
            new_line=""
            new_line2=""
            new_line3=""
            if match1:
                new_line =F'\n  keyboardJson += R"([{{ "text" : "{new_attribute_desc} )";'
                new_line2=F'  keyboardJson += (configItems.{new_attribute_name}?"ON\\u2705":"OFF\\u274C");'
                new_line3=F'  keyboardJson += R"(", "callback_data" : "/{new_attribute_name}" }}],)";'
            
            if match2:
                new_line =F'          }}else if (text == "/{new_attribute_name}") {{'
                new_line2=F'            configItems.{new_attribute_name} = !configItems.{new_attribute_name};'
            
            if match3:
                new_line =F'          welcome += "\\t {new_attribute_name} | {new_attribute_desc}\\n";'
            
            ############################
            if new_line!="":
                print (new_line)
                lines.append(new_line+"\n")
            if new_line2!="":
                print (new_line2)
                lines.append(new_line2+"\n")
            if new_line3!="":
                print (new_line3)
                lines.append(new_line3+"\n")
            ############################
        new_telegramUtils_h=open(F"{telegramUtils_h}_new","w")
        for line in lines:
            #print (line)
            new_telegramUtils_h.write(line)
        new_telegramUtils_h.close()
        
    if os.path.exists(F"{telegramUtils_h}_orig"):
        os.unlink(F"{telegramUtils_h}_orig")
    os.rename(telegramUtils_h,F"{telegramUtils_h}_orig")
    os.rename(F"{telegramUtils_h}_new",telegramUtils_h)

## Changes in webPages.h

In [5]:
import re 

def changeWebPages(anchor_attribute,new_attribute_name,new_attribute_type,new_attribute_desc):

    with open(webPages_h,"r") as webPagesH:
        lines=[]
        for line in webPagesH:
            #print (line)
            ############################
            #configItems.vFlip=(args.hasArg("XvFlip")?true:false);
            lookfor=r'configItems.vFlip=(args.hasArg("XvFlip")?true:false);'
            lookfor=lookfor.replace("vFlip",anchor_attribute)
            match1 = re.search(re.escape(lookfor), line)

            #aux["XvFlip"].as<AutoConnectCheckbox>().checked=configItems.vFlip;
            lookfor=r'aux["XvFlip"].as<AutoConnectCheckbox>().checked=configItems.vFlip;'
            lookfor=lookfor.replace("vFlip",anchor_attribute)
            match2 = re.search(re.escape(lookfor), line)

            #{ "name": "XvFlip", "type": "ACCheckbox", "value": "", "labelPosition": "AC_Infront" , "label": "Vertical Flip", "checked": false, "global": true },
            lookfor=r'{ "name": "XvFlip","type": "ACCheckbox","value": "","labelPosition": "AC_Infront" ,"label": "Vertical Flip","checked": false,"global": true},'
            lookfor=lookfor.replace("vFlip",anchor_attribute)
            match3 = re.search(re.escape(lookfor), line)
                
            ############################
            lines.append(line)
            new_line=""
            new_line2=""
            new_line3=""
            if match1:
                new_line =F'    configItems.{new_attribute_name}=(args.hasArg("X{new_attribute_name}")?true:false);'
            
            if match2:
                new_line =F'    aux["X{new_attribute_name}"].as<AutoConnectCheckbox>().checked=configItems.{new_attribute_name};'

            if match3:
                new_line =F'    {{ "name": "X{new_attribute_name}", "type": "ACCheckbox", "value": "", "labelPosition": "AC_Infront" , "label": "{new_attribute_desc}", "checked": false, "global": true }},'
            
            
            ############################
            if new_line!="":
                print (new_line)
                lines.append(new_line+"\n")
            if new_line2!="":
                print (new_line2)
                lines.append(new_line2+"\n")
            if new_line3!="":
                print (new_line3)
                lines.append(new_line3+"\n")
            ############################
        new_webPages_h=open(F"{webPages_h}_new","w")
        for line in lines:
            #print (line)
            new_webPages_h.write(line)
        new_webPages_h.close()

    if os.path.exists(F"{webPages_h}_orig"):
        os.unlink(F"{webPages_h}_orig")
    os.rename(webPages_h,F"{webPages_h}_orig",)
    os.rename(F"{webPages_h}_new",webPages_h)

## Assembly : add multiple attributes

In [9]:

attsToAdd = [ 
    {
        new_attribute_name: "motionDetectFrom",
        new_attribute_type: "int",
        new_attribute_desc: "MotionDetect active from",
    }
    ,{
        new_attribute_name: "motionDetectTo",
        new_attribute_type: "int",
        new_attribute_desc: "MotionDetect active from",
    }
    ,{
        new_attribute_name: "faceDetect",
        new_attribute_type: "boolean",
        new_attribute_desc: "Alert on detecting a face",
    }
    ,{
        new_attribute_name: "takeVideo",
        new_attribute_type: "boolean",
        new_attribute_desc: "Take a Video",
    }
    ,{
        new_attribute_name: "longitude",
        new_attribute_type: "float",
        new_attribute_desc: "Location Longitude",
    }
    ,{
        new_attribute_name: "latitude",
        new_attribute_type: "float",
        new_attribute_desc: "Location Latitude",
    }
    ,{
        new_attribute_name: "altitude",
        new_attribute_type: "float",
        new_attribute_desc: "Location Altitude",
    }
    ,{
        new_attribute_name: "snapSunrise",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Sunrise",
    }
    ,{
        new_attribute_name: "snapNoon",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Noon",
    }
    ,{
        new_attribute_name: "snapAfterNoon",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on After Noon",
    }
    ,{
        new_attribute_name: "snapLateAfterNoon",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Late After Noon",
    }
    ,{
        new_attribute_name: "snapSunset",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Sunset",
    }
    ,{
        new_attribute_name: "snapEvening",
        new_attribute_type: "boolean",
        new_attribute_desc: "Snap a photo on Evening",
    }
    ,{
        new_attribute_name: "jpegQulaity",
        new_attribute_type: "int",
        new_attribute_desc: "Jpeg Quality 1-62[10-12]",
    }
    ,{
        new_attribute_name: "brightness",
        new_attribute_type: "int",
        new_attribute_desc: "PhotoBrightness 1-62[10-12]",
    }
    ,{
        new_attribute_name: "whiteBalance",
        new_attribute_type: "int",
        new_attribute_desc: "PhotoBrightness 1-62[10-12]",
    }
    ,{
        new_attribute_name: "sendEmail",
        new_attribute_type: "boolean",
        new_attribute_desc: "Alert by Email.",
    }
    ,{
        new_attribute_name: "adminEmail",
        new_attribute_type: "String",
        new_attribute_desc: "Email 1-dont use your real email, create a new one",
    }
    ,{
        new_attribute_name: "userEmail",
        new_attribute_type: "String",
        new_attribute_desc: "Email 2.",
    }
    ,{
        new_attribute_name: "sMTPServer",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP Email Server",
    }
    ,{
        new_attribute_name: "sMTPUsername",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP Email username",
    }
    ,{
        new_attribute_name: "sMTPPassword",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP Email Password",
    }
    ,{
        new_attribute_name: "sMTPPort",
        new_attribute_type: "int",
        new_attribute_desc: "SMTP Email Port",
    }
    ,{
        new_attribute_name: "sMTPTLS",
        new_attribute_type: "String",
        new_attribute_desc: "SMTP TLS/SSL Required",
    }
]

for att in attsToAdd:
    print(att)
    changePersist(anchor_attribute,att[new_attribute_name],att[new_attribute_type],att[new_attribute_desc])
    changeTelegram_utils(anchor_attribute,att[new_attribute_name],att[new_attribute_type],att[new_attribute_desc])
    changeWebPages(anchor_attribute,att[new_attribute_name],att[new_attribute_type],att[new_attribute_desc])

{'useBuzzer': 'useBuzzer', 'boolean': 'boolean', 'Buzzer on motion Detect': 'trigger buzzer on motion detect'}
  boolean useBuzzer;
  .useBuzzer = true,
    prefs.remove("useBuzzer");
    ci.useBuzzer = prefs.getBool("useBuzzer",configItems.useBuzzer);
    if (prefs.getBool("useBuzzer")!=ci->useBuzzer)
      { prefs.putBool("useBuzzer",ci->useBuzzer); bDirty=true; }
  result += prefix+"useBuzzer            "+sep+"";
  result += (ci->useBuzzer ? String("true") : String("false"))  + suffix;

  keyboardJson += R"([{ "text" : "trigger buzzer on motion detect )";
  keyboardJson += (configItems.useBuzzer?"ON\u2705":"OFF\u274C");
  keyboardJson += R"(", "callback_data" : "/useBuzzer" }],)";
          }else if (text == "/useBuzzer") {
            configItems.useBuzzer = !configItems.useBuzzer;
          welcome += "\t useBuzzer | trigger buzzer on motion detect\n";
    configItems.useBuzzer=(args.hasArg("XuseBuzzer")?true:false);
    aux["XuseBuzzer"].as<AutoConnectCheckbox>().checked=configIt